# Data Preprocessing

## Overview

The raw data from Eraslan et al. is a tab separated table containing gene names, related Ensembl IDs and measured or calculated values for mRNA abundance, protein abundance and protein-to-mRNA ratio.

In the first Jupyter cells the data is roughly explored. After that the relevant values for the up coming analysis are extracted.

Some problems arose:
- Not all transcript IDs seem to be the current canonical form of transcript for a particular gene. In fact the given IDs point to transcripts that do not translate (in most cases). The gene needs to be identified in that case and the up to date transcript ID resolved.
- BUT there are still some transcript IDs left that are annotate with "nonsense mediated decay". These will be thrown out as they do not successfully translate. 
- The Genecode data set contains duplicate files for some of the given transcript IDs. They can be easily filtered using a regex. The duplicates are the exact same files with different names.

In [ ]:
# library dependencies
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import re

## Reading the data

In [ ]:
# raw data file and path
datafile = '../data/Eraslan-EV3.tsv'

# sanity check if the file exists
if not Path(datafile).is_file():
    print('Data file not found!')

## Exploring the data

In [ ]:
# reading the data into a dataframe and looking at the first entries
df = pd.read_csv(datafile, sep='\t')
df

In [ ]:
# looking at the columns
df.info()

In [ ]:
df.describe()

Between ~3000 and ~4000 values in each of the 11575 rows are NA

## Extracting the relevant columns

Only the _EnsemblTranscriptID_ and _PTR_ values per tissue are necessary for training the network.

In [ ]:
df2 = df[['EnsemblTranscriptID'] + [ col for col in df.columns if col.endswith('_PTR') ]].copy()
df2

Cross referencing the transcript IDs with BED and Fasta files from the gencode data set (43).

The path set below expects the gencode repo to be relative to this notebook.

In [ ]:
# raw data paths
gencode_path = '../../GENCODE43/protein_coding/'
bed = Path(gencode_path) / 'BED6__protein_coding_strict/'
fa = Path(gencode_path) / 'FA_protein_coding_strict_mRNA/'

# file names look like this
# for the BED file : ENST00000370801.8.bed
# for the Fasta file : ENST00000370801.8:0-6412.fasta
# .8 denotes the current Ensemble version
# :0-6412 is the nucleotide length

# count of processed transcript IDs
count_all = 0
# success count
count_found = 0
# multiple files found for transcript
count_multi = 0

# extend the dataframe
# number of files found
df2['bed_files'] = 0
df2['fa_files'] = 0
# file path and name
df2['bed'] = ''
df2['fa'] = ''

# checking if all the transcript Fasta and BED files per transcript exist
for tid in df2['EnsemblTranscriptID']:
    # inclrease over all count
    count_all += 1

    # list and count files
    bed_file_list = list(bed.glob(tid + '*.bed'))
    bed_file_count = len(bed_file_list)
    fa_file_list = list(fa.glob(tid + '*.fasta'))
    fa_file_count = len(fa_file_list)

    # update dataframe
    df2.loc[ df2['EnsemblTranscriptID'] == tid, 'bed_files'] = bed_file_count
    df2.loc[ df2['EnsemblTranscriptID'] == tid, 'fa_files'] = fa_file_count

    # check BED and Fasta file count
    if bed_file_count == 1 and fa_file_count == 1:
        # exctly one BED and FA file
        
        # increase hit count
        count_found += 1
        
        # update file name information
        df2.loc[ df2['EnsemblTranscriptID'] == tid, 'bed'] = str(bed_file_list[0])
        df2.loc[ df2['EnsemblTranscriptID'] == tid, 'fa'] = str(fa_file_list[0])
    elif bed_file_count == 2 and fa_file_count == 2:
        # special case where there are duplicate files
        print(tid, 'more than one BED/Fasta file present. selecting')

        # increase hit count
        count_found += 1
        count_multi += 1

        # find correct BED file and update table
        for f in bed_file_list:
            temp_bed_file = str(f)
            if re.search(r'.*ENST\d+\.\d+.bed', temp_bed_file):
                df2.loc[ df2['EnsemblTranscriptID'] == tid, 'bed'] = temp_bed_file
                print('   ', temp_bed_file)

        # find correct Fasta file and update table
        for f in fa_file_list:
            temp_fa_file = str(f)
            if re.search(r'.*ENST\d+\.\d+:\d+-\d+.fasta', temp_fa_file):
                df2.loc[ df2['EnsemblTranscriptID'] == tid, 'fa'] = temp_fa_file
                print('   ', temp_fa_file)

        # update file count in table
        df2.loc[ df2['EnsemblTranscriptID'] == tid, 'bed_files'] = 1
        df2.loc[ df2['EnsemblTranscriptID'] == tid, 'fa_files'] = 1
    else:
        # everything else ends up here
        print(tid, 'bed count:', bed_file_count, 'fa count:', fa_file_count, 'bed files:', bed_file_list, 'fa files:', fa_file_list)

print('searched for', count_all, 'and found', count_found)
print('found multiple files for', count_multi, 'transcripts')
print('missing or otherwise off:', count_all - count_found)

In [ ]:
# entries with two transcript files per entry
# the gencod data set contains a couple of transcript files with multiple different names
# file countent is exaclty the same
# this has been corrected in the previous cell so this sanity check should reveal 0 rows
df2.loc[ df2['bed_files'] == 2 ]

In [ ]:
# check again the number of missing files (with a count of 0 in the bed_file column)
df2.loc[ df2['bed_files'] == 0, 'EnsemblTranscriptID' ].count()

In [ ]:
# show some of the IDs with missing files
df2.loc[ df2['bed_files'] == 0, 'EnsemblTranscriptID' ]

Sampling the IDs for which files are missing showed that the transcript is either deprecated or not the canonical form in the gencode data set (43). For 11 IDs there are no corresponding transcript entries any more.

This is a natural evolution since the Eraslan et al. research took place 2019 the underlying data in the Ensembl database got updated with current research results.

The following cells query the Ensembl web site directly for the transcript IDs in question as it's the fastest way to resolve this issue.

In [ ]:
def find_new_transcript(req):
    """Method to extract a specific transcript ID from an HTML document.

    Keyword Arguments:
    req -- Python request object

    Returns:
    A string either empty or containing the transcript ID.
    """
    # parse the HTML document
    soup = BeautifulSoup(r.content, 'html.parser')
    # check if a specific table exists
    if soup.find(id='transcripts_table'):
        # if so extract the transcript ID
        href = soup.find(id='transcripts_table').tbody.td.a.attrs['href']
        transcript = re.sub(r'.*(ENST0\d+)', r'\1', href)
        print('   Current canonical transcript is', transcript)
    else:
        # if not return an empty string
        transcript = ''
        print('   No current transcript found!')

    return transcript

def check_files_and_update_df(transcript):
    """Cross reference the transcript ID with files in the gencode data set
    (bad hack as it uses variables globally defined at the beginning of this notebook!)

    Keyword Arguments:
    transcript -- the transcript ID string
    """
    # search and count files with a given name
    bed_file_list = list(bed.glob(transcript + '*.bed'))
    bed_files = len(bed_file_list)
    fa_file_list = list(fa.glob(transcript + '*.fasta'))
    fa_files = len(fa_file_list)

    # check how many files were found
    if bed_files == 1 and fa_files == 1:
        # if it's 1 everything is perfect
        print('   FA and BED files found. Updating dataframe with current information')
        # update dataframe
        df2.loc[ df2['EnsemblTranscriptID'] == tid, 'bed_files' ] = bed_files
        df2.loc[ df2['EnsemblTranscriptID'] == tid, 'fa_files' ] = fa_files
        df2.loc[ df2['EnsemblTranscriptID'] == tid, 'EnsemblTranscriptID' ] = transcript
    else:
        # if there are many manual processing is needed
        print('   FA and BED file count invalid. File lists', bed_file_list, fa_file_list)

In [ ]:
# loop over all transcript IDs without a transcript file associated with it
for tid in df2.loc[ df2['bed_files'] == 0, 'EnsemblTranscriptID' ]: #.head(2):
    print('processing', tid)
    # Ensembl URL for resolving the given transcript ID
    url = 'https://www.ensembl.org/Homo_sapiens/Transcript/Idhistory?t=' + tid
    # retrieve the document
    r = requests.get(url)
    # parse the document
    soup = BeautifulSoup(r.content, 'html.parser')
    # check for specific strings in the page
    if re.search(r'This transcript is not in the current gene set', soup.get_text()):
        # transcript is deprecated so extract the corresponding gene and get the canonical transcript ID
        href = soup.td.next_sibling.a.attrs['href']
        gene = re.sub(r'.*(ENSG0\d+)', r'\1', href)
        print('   Transcript is deprecated, resolved gene is', gene)

        # Ensembl URL to resolve a gene ID
        url = 'https://www.ensembl.org/Homo_sapiens/Gene/Idhistory?g=' + gene
        r = requests.get(url)
        transcript = find_new_transcript(r)
        check_files_and_update_df(transcript)
    elif re.search(r'Show transcript table', soup.get_text()):
        # the transcript is not the current canonical version and needs updating
        transcript = find_new_transcript(r)
        check_files_and_update_df(transcript)
    else:
        print('   Some other error occured for this transcript')

In [ ]:
# check if there are still entries with unresolved transcript files
# spoiler, there are 11
df2.loc[ df2['bed_files'] == 0, 'EnsemblTranscriptID' ].count()

In [ ]:
# show the 11 IDs with missing transcript files
df2.loc[ df2['bed_files'] == 0 ]

Even though these transcript IDs will have resolved in 2019 when the paper using this data was published the current database does not resolve these IDs any more (due to more up to date research results).

Being brave those entries will be deleted.

In [ ]:
# remove the 11 missing entries
df2.drop(df2[df2['bed_files'] == 0].index, inplace=True)

In [ ]:
# verify the entries are gone from the dataframe
df2.loc[ df2['bed_files'] == 0 ]

In [ ]:
# write current pre processed table to file
datafile = '../data/preproc.csv'
df2.to_csv(datafile, index=False)